In [1]:
%load_ext autoreload
%autoreload 2
from specxploreImporting import importing
import matchms
from matchms.importing import load_from_mgf
from matchms.exporting import save_as_mgf
from ms2query.run_ms2query import download_zenodo_files, run_complete_folder
from ms2query.ms2library import create_library_object_from_one_dir
import os
import pandas as pd
import numpy as np
from typing import List

In [2]:
models_and_library_folder_pos = os.path.join("data_and_output", "models", "ms2query_models_and_library_pos")
raw_experimental_mgf_filename = os.path.join("data_and_output", "wheat_data", "MonoisotopicMS2Feature_pos_mostIntense.mgf")
raw_standards_mgf_filename = os.path.join("data_and_output", "wheat_data", "phophe_pos.mgf")
processed_experimental_spectra_filename = os.path.join("data_and_output", "wheat_data", "spectra_experimental.mgf")
processed_standard_spectra_filename = os.path.join("data_and_output", "wheat_data", "spectra_standards.mgf")
csv_experimental_filename = os.path.join("data_and_output", "wheat_data", "wheat_data_ms2query_out_experimental.csv")
csv_standards_filename = os.path.join("data_and_output", "wheat_data", "wheat_data_ms2query_out_standard.csv")
scores_s2v_filename = os.path.join("data_and_output", "wheat_data", "s2v.npy")
scores_ms2ds_filename = os.path.join("data_and_output", "wheat_data", "ms2ds.npy")
scores_cos_filename = os.path.join("data_and_output", "wheat_data", "cos.npy")

# Process Raw Data


In [3]:
spectra_experimental = list(load_from_mgf(raw_experimental_mgf_filename))
spectra_experimental = importing.clean_spectra(spectra_experimental)


spectra_standards = list(load_from_mgf(raw_standards_mgf_filename))
spectra_standards = importing.clean_spectra(spectra_standards)

if False:
    save_as_mgf(spectra_experimental, processed_experimental_spectra_filename)
    save_as_mgf(spectra_standards, processed_standard_spectra_filename)

2023-04-18 10:52:56,920:WARNING:matchms:add_compound_name:No compound name found in metadata.
2023-04-18 10:52:56,923:WARNING:matchms:add_compound_name:No compound name found in metadata.
2023-04-18 10:52:56,924:WARNING:matchms:add_compound_name:No compound name found in metadata.
2023-04-18 10:52:56,926:WARNING:matchms:add_compound_name:No compound name found in metadata.
2023-04-18 10:52:56,927:WARNING:matchms:add_compound_name:No compound name found in metadata.
2023-04-18 10:52:56,928:WARNING:matchms:add_compound_name:No compound name found in metadata.
2023-04-18 10:52:56,929:WARNING:matchms:add_compound_name:No compound name found in metadata.
2023-04-18 10:52:56,930:WARNING:matchms:add_compound_name:No compound name found in metadata.
2023-04-18 10:52:56,931:WARNING:matchms:add_compound_name:No compound name found in metadata.
2023-04-18 10:52:56,932:WARNING:matchms:add_compound_name:No compound name found in metadata.
2023-04-18 10:52:56,933:WARNING:matchms:add_compound_name:No

# Run Classification using ms2query

In [4]:
# WARNING
# THIS DOES WORK, BUT AUTOMATICALLY RUNS BOTH STANDARDS AND EXPERIMENTAL SPECTRA IF IN SAME FOLDER
# UPDATE TO MS2QUERY LATEST VERSION TO ALLOW SINGLE FILE INPUT AND OUTPUT
# ONCE UPDATED, FILENAMES CAN ALSO BE SPECIFIED....
if False:
    ms2library = create_library_object_from_one_dir(models_and_library_folder_pos)
    importing.run_single_file(ms2library, processed_standard_spectra_filename, str(csv_standards_filename))
    importing.run_single_file(ms2library, processed_experimental_spectra_filename, str(csv_experimental_filename))

# Merge Classes Tables and Spectra lists

In [4]:
classes_experimental = pd.read_csv(csv_experimental_filename)
classes_experimental = importing.expand_ms2query_results_table(classes_experimental, len(spectra_experimental))
classes_standards = pd.read_csv(csv_standards_filename)
classes_standards = importing.expand_ms2query_results_table(classes_standards, len(spectra_standards))

In [5]:
classes_experimental["is_standard"] = False
classes_experimental["exp_metadata"] = None
classes_standards["is_standard"] = True
classes_standards["standard_metadata"] = None
all_class_table = pd.merge(classes_experimental, classes_standards, how = 'outer').reset_index()
all_class_table["specxplore_id"] = all_class_table.index
all_class_table

,index,query_spectrum_nr,ms2query_model_prediction,precursor_mz_difference,precursor_mz_query_spectrum,precursor_mz_analog,inchikey,spectrum_ids,analog_compound_name,retention_time,...,cf_subclass,cf_direct_parent,npc_class_results,npc_superclass_results,npc_pathway_results,source_spectrum_mgf_iloc,is_standard,exp_metadata,standard_metadata,specxplore_id
0,0,1,0.3775,123.0081,105.0339,228.042,ZCGLNFAUVVHTQO,276075,3-[(4-chlorobenzoyl)amino]propanoic acid,NaN,...,"Amino acids, peptides, and analogues",Beta amino acids and derivatives,NaN,NaN,NaN,0,False,None,None,0
1,1,2,0.7120,0.0007,113.0237,113.023,IHCCAYCGZOLTEU,303482,3-FUROIC ACID,NaN,...,Furoic acid and derivatives,Furoic acids,NaN,NaN,NaN,1,False,None,None,1
2,2,3,0.7416,0.0001,120.0809,120.081,LPAGFVYQRIESJQ,313913,Indolin,NaN,...,Indolines,Indolines,Carboline alkaloids,Tryptophan alkaloids,Alkaloids,2,False,None,None,2
3,3,4,0.7399,0.0006,122.0966,122.096,UFFBMTHBGFGIHF,274723,"2,6-Dimethylaniline",NaN,...,Xylenes,m-Xylenes,NaN,NaN,NaN,3,False,None,None,3
4,4,5,0.4090,310.2715,125.0235,435.295,SCABKEBYDRTODC,265703,bis[2-(2-butoxyethoxy)ethyl] hexanedioate,NaN,...,Fatty acid esters,Fatty acid esters,Triacylglycerols,Glycerolipids,Fatty acids,4,False,None,None,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,604,21,0.4887,95.0433,579.1483,484.105,CNURKUNYCXCBEK,283054,karakin,NaN,...,Tricarboxylic acids and derivatives,Tricarboxylic acids and derivatives,NaN,NaN,NaN,20,True,NaN,NaN,604
605,605,22,0.9419,0.0004,579.1494,579.149,XFZJEEAOWLFHDH,24733,Procyanidin B2,NaN,...,Biflavonoids and polyflavonoids,Biflavonoids and polyflavonoids,Proanthocyanins,Flavonoids,Shikimates and Phenylpropanoids,21,True,NaN,NaN,605
606,606,23,0.9387,0.0002,581.1858,581.186,HXTFHSYLYXVTHC,13249,"5-hydroxy-2-(4-hydroxyphenyl)-7-[(2S,3R,4S,5S,...",NaN,...,Flavonoid glycosides,Flavonoid-7-O-glycosides,Flavanones,Flavonoids,Shikimates and Phenylpropanoids,22,True,NaN,NaN,606
607,607,24,0.9387,0.0002,611.1958,611.196,ARGKVCXINMKCAZ,18761,"(2S)-7-[(2S,3R,4S,5S,6R)-4,5-dihydroxy-6-(hydr...",NaN,...,Flavonoid glycosides,Flavonoid-7-O-glycosides,Flavanones,Flavonoids,Shikimates and Phenylpropanoids,23,True,NaN,NaN,607


In [6]:
all_spectra = spectra_experimental + spectra_standards # list addition

In [8]:
if False:
    scores_s2v = importing.compute_similarities_s2v(all_spectra, models_and_library_folder_pos)
    scores_cos = importing.compute_similarities_cosine(all_spectra, cosine_type="ModifiedCosine")
    scores_ms2ds = importing.compute_similarities_ms2ds(all_spectra, models_and_library_folder_pos)
    np.save(scores_ms2ds_filename, scores_ms2ds, allow_pickle=False)
    np.save(scores_s2v_filename, scores_s2v, allow_pickle=False)
    np.save(scores_cos_filename, scores_cos, allow_pickle=False)

In [8]:
scores_s2v = np.load(scores_s2v_filename)
scores_ms2ds = np.load(scores_ms2ds_filename)
scores_cos = np.load(scores_cos_filename)

scores_heuristic = np.maximum(scores_ms2ds, scores_cos)
scores_ms2ds = scores_heuristic # <-- overwriting ms2ds score here!!!

# Run Kmedoid and tsne

In [9]:
k_values = [10,20,30,40,50,75,100, 125, 150, 175, 200,250, 300, 400,500]
random_seeds = [int(np.random.randint(0,1000)) for _ in k_values]
similarities = scores_ms2ds
distances = importing.convert_similarity_to_distance(similarities)
kmedoid_list = importing.run_kmedoid_grid(distances, k_values, random_seeds)
importing.render_kmedoid_fitting_results_in_browser(kmedoid_list)

In [10]:
perplexity_values = [x for x in range(5, 55, 5)]
random_seeds = [int(np.random.randint(0,1000)) for _ in perplexity_values]
tsne_list_pos = importing.run_tsne_grid(distances, perplexity_values, random_seeds)
importing.render_tsne_fitting_results_in_browser(tsne_list_pos)

# Merge into specxplore object

In [11]:
%load_ext autoreload
%autoreload 2
import specxplore.specxplore_data
from specxplore.specxplore_data import specxplore_data, Spectrum
is_standard = np.array(all_class_table["is_standard"])
spec_classes = all_class_table[['cf_kingdom', 'cf_superclass', 'cf_class', 'cf_subclass',
       'cf_direct_parent', 'npc_class_results', 'npc_superclass_results',
       'npc_pathway_results']]
mz = [spec.get("precursor_mz") for spec in all_spectra]
# KmedoidGridEntry(k, cluster_assignments, score, random_states[idx])
kclass_table = pd.DataFrame()
for elem in kmedoid_list:
    kclass_table[elem.k] = elem.cluster_assignments
classes = pd.concat([spec_classes, kclass_table], axis=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
tsnedf = pd.DataFrame({"x" : tsne_list_pos[8].x_coordinates, "y" : tsne_list_pos[8].y_coordinates})
specxplore_id = np.array(all_class_table["specxplore_id"])

def convert_matchms_spectra_to_specxplore_spectrum(spectra = List[matchms.Spectrum]) -> List[Spectrum]:
  spectra_converted = [
      specxplore.specxplore_data.Spectrum(spec.peaks.mz, float(spec.get("precursor_mz")), idx, spec.peaks.intensities) 
      for idx, spec in enumerate(all_spectra)]
  return spectra_converted

spectra_converted = convert_matchms_spectra_to_specxplore_spectrum(all_spectra)

wheat_data_specxplore = specxplore_data(
  scores_ms2ds,scores_s2v, scores_cos, tsnedf,  classes,
  is_standard, spectra_converted, mz, specxplore_id)

In [21]:
wheat_data_specxplore.spectra[0]

Spectrum(mass_to_charge_ratios=array([ 51.02379,  53.0393 ,  70.08131,  72.93778,  77.03909,  78.03838,
        79.05466,  82.55311,  90.94814,  95.04961, 103.05476, 105.02575,
       105.03387, 105.04509, 105.07024, 114.49272]), precursor_mass_to_charge_ratio=105.03386, identifier=0, intensities=array([0.03      , 0.21578947, 0.01315789, 0.06315789, 0.12105263,
       0.01263158, 0.02631579, 0.01263158, 0.02315789, 1.        ,
       0.01526316, 0.01842105, 0.57894737, 0.27368421, 0.06842105,
       0.01210526]), mass_to_charge_ratio_aggregate_list=(), intensity_aggregate_list=(), is_binned_spectrum=False, is_neutral_loss=False)

In [13]:
import pickle
with open('data_and_output/wheat_data/wheat_data_specxplore_v5.pickle', 'wb') as file:
  pickle.dump(wheat_data_specxplore, file)